In [69]:
import duckdb
import pandas as pd

In [70]:
#download data
exit_velocity_data = pd.read_csv('data/mlb-batter-exit-velocity.csv')

# run sql file to create tables
con = duckdb.connect('baseball_data.db').execute(open('baseball_data.sql').read())

# Create a DataFrame from a query
batting = con.execute("SELECT * FROM batting").df()

# Close the connection to the database
con.close()


In [71]:
exit_velocity_data['name'] = exit_velocity_data['player'].str.split(', ').str[1] + ' ' + exit_velocity_data['player'].str.split(', ').str[0].str.strip()

In [72]:
batting['name'] = batting['name'].str.strip()

In [81]:
# join the two dataframes on name and year
df = batting.merge(exit_velocity_data, how='inner', left_on=['name', 'Year'], right_on=['name', 'year'])
dups = df[df.duplicated(subset=['name'],)]
dups.sort_values('name')


,Rk,name,Age,Team,Lg,G,PA,AB,R,H,...,ground_ball_ev,max_distance,average_distance,average_homerun,hard_hit_95mph+,hard_hit_percentage,hard_hit_swing_percentage,total_barrels,barrels_batted_balls_percentage,barrels_plate_appearance_percentage
610,905,AJ Pollock,31,LAD,NL,86,342,308,49,82,...,88.9,429,173,398.0,95,39.9,16.2,19,8.0,5.6
481,888,AJ Pollock,30,ARI,NL,113,460,413,61,106,...,84.8,455,183,405.0,130,40.6,15.7,32,10.0,7.0
348,871,AJ Pollock,29,ARI,NL,112,466,425,73,113,...,83.5,433,169,406.0,135,38.4,18.3,20,5.6,4.3
737,447,AJ Pollock,32,LAD,NL,55,210,196,30,54,...,83.8,428,187,403.0,66,43.1,18.5,16,10.5,7.6
862,967,AJ Pollock,33,LAD,NL,117,422,384,53,114,...,84.8,430,176,405.0,145,47.4,18.7,34,11.1,8.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
827,499,Yuli Gurriel,37,HOU,AL,143,605,530,83,169,...,91.3,413,161,384.0,199,42.0,20.3,16,3.4,2.6
437,465,Yuli Gurriel,34,HOU,AL,136,573,537,70,156,...,91.0,427,165,378.0,176,36.6,18.4,9,1.9,1.6
159,330,Yunel Escobar,33,LAA,AL,132,567,517,68,157,...,88.3,413,129,398.0,165,36.1,17.3,13,2.8,2.3
288,329,Yunel Escobar,34,LAA,AL,89,381,350,43,96,...,86.3,438,147,412.0,117,39.1,18.4,15,5.0,3.9


In [74]:
#total bases
df['TB'] = df['H'] + df['2B'] + 2 * df['3B'] + 3 * df['HR']

#Bat Avg
df['AVG'] = (df['H'] / df['AB']).round(3) #, where=df['AB'] != 0

#on base
numerator_obp = df['H'] + df['BB'] + df['HBP']
denominator_obp = df['AB'] + df['BB'] + df['HBP'] + df['SF']
obp_calculation = numerator_obp / denominator_obp
df['OBP'] = obp_calculation.round(3)# , where=denominator_obp != 0

#slugging
slugging_calculation = df['TB'] / df['AB']
df['SLG'] = slugging_calculation.round(3)# , where=df['AB'] != 0

#on base + slugging 
df['OPS'] = (obp_calculation + slugging_calculation).round(3)

print("DataFrame with new stats columns (AVG, OBP, SLG, OPS):")
print(df[['name', 'Year', 'AB', 'H', 'TB', 'AVG', 'OBP', 'SLG', 'OPS']].head())

DataFrame with new stats columns (AVG, OBP, SLG, OPS):
            name  Year   AB    H   TB    AVG    OBP    SLG    OPS
0     José Abreu  2015  613  178  308  0.290  0.347  0.502  0.850
1     Nick Ahmed  2015  421   95  151  0.226  0.275  0.359  0.634
2    Jose Altuve  2015  638  200  293  0.313  0.353  0.459  0.812
3   Elvis Andrus  2015  596  154  213  0.258  0.309  0.357  0.667
4  Nolan Arenado  2015  616  177  354  0.287  0.323  0.575  0.898


In [ ]:
# batting = con.execute("SELECT * FROM batting").df()

stats_to_pivot = ['G', 'PA', 'AB', 'R', 'H', 'HR', 'RBI']


# - index: The column(s) to remain as rows (Player/Name)
# - columns: The column whose values will become the new column names (Year)
# - values: The columns whose values will populate the new wide columns (Selected Stats)
wide_batting_df = batting.pivot_table(
    index=['name'],           # one row per player
    columns='Year',           # Use Year values  as new columns
    values=stats_to_pivot,    # stats to pivot
    aggfunc='sum'             # Use 'sum' to aggregate if a player has multiple entries in a year,
).reset_index()

wide_batting_df['name'] = wide_batting_df['name'].str.strip()
wide_batting_df['name'] = wide_batting_df['name'].str.replace('*', '', regex=False)
wide_batting_df['name'] = wide_batting_df['name'].str.replace('#', '', regex=False)

wide_batting_df.to_csv('wide_batting_stats.csv')

KeyError: Index(['name'], dtype='object')